In [146]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0.5):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.n_layers, batch_size, self.hidden_size).to(device))


In [147]:
import string
char_to_num = {char: i for i, char in enumerate(string.ascii_lowercase, 1)}

# 字符到数字的映射
char_to_num = {char: i for i, char in enumerate(string.ascii_lowercase, 1)}
char_to_num['<pad>'] = 0  # 填充符
char_to_num['-'] = 27  # -
char_to_num['<eos>'] = 28  # 结束符
char_to_num['<bos>'] = 29  # 开始符（如果使用）

num_to_char = {i: j for j, i in char_to_num.items()}

def tokenize(text, bidirectional=False):
    # 先全部变成小写，把大写加进来可能会出现
    lowercase_text = text.lower()
    # 27用来表示结束，如果需要可以在头上加上0，这样就可以随意给出一段文字然后来生成开始和结尾
    numbers = [char_to_num[char] for char in text if char in char_to_num] + [28]
    if bidirectional == True:
        numbers = [29] + numbers
    return numbers

def reverse_tokenize(numbers, bidirectional=False):
    # 如果是双向模式，跳过第一个数字（29）
    start_index = 1 if bidirectional and numbers[0] == 29 else 0

    # 转换数字到字符，直到遇到28
    characters = []
    for number in numbers[start_index:]:
        if number == 28:
            break
        characters.append(num_to_char[number])


    # 将字符列表转换为字符串
    return ''.join(characters)


In [148]:
bidirectional = False
batch_size = 32
learning_rate = 0.01
input_size = 29 + int(bidirectional)
hidden_size = 100
output_size = 29 + int(bidirectional)
n_layers = 1
dropout = 0.5
n_epochs = 10

model = RNNModel(input_size, hidden_size, output_size, n_layers, dropout)
loss_function = nn.CrossEntropyLoss(ignore_index=char_to_num['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [149]:
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from torch.nn.utils.rnn import pad_sequence

# 读取和处理数据
def load_data(filename):
    names = []
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            name = line.strip()
            name = tokenize(name, bidirectional=bidirectional)
            names.append(torch.tensor(name, dtype=torch.long))
    return names

names = load_data('names.txt')
f_names = load_data('female.txt')
m_names = load_data('male.txt')
dataset = f_names + m_names 


lengths = [len(sequence) for sequence in dataset]
padded_dataset = pad_sequence([torch.tensor(sequence, dtype=torch.long) for sequence in dataset], \
                              batch_first=True, padding_value=0)
data_loader = DataLoader(padded_dataset, batch_size=batch_size, shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

C:\Users\HenryLi\AppData\Local\Temp\ipykernel_6452\3107712702.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_dataset = pad_sequence([torch.tensor(sequence, dtype=torch.long) for sequence in dataset], \


In [150]:
model.train()
for epoch in range(n_epochs):
    for batch in data_loader:
        hidden = model.init_hidden(batch.shape[0])
        input, target = batch[:, :-1], batch[:, 1:]    # 移除最后一个字符作为输入，第二个字符到最后一个字符作为目标
        input, target = input.to(device), target.to(device)
        optimizer.zero_grad()
        output, hidden = model(input, hidden)
        hidden = tuple([each.data for each in hidden])  # 分离隐藏状态
        # 计算损失
        loss = loss_function(output.reshape(-1, output_size), target.reshape(-1))
        loss.backward()
        optimizer.step()

        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 0/10............. Loss: 3.3389
Epoch: 0/10............. Loss: 3.1496
Epoch: 0/10............. Loss: 2.9268
Epoch: 0/10............. Loss: 2.7038
Epoch: 0/10............. Loss: 2.5970
Epoch: 0/10............. Loss: 2.6064
Epoch: 0/10............. Loss: 2.8055
Epoch: 0/10............. Loss: 2.3470
Epoch: 0/10............. Loss: 2.3770
Epoch: 0/10............. Loss: 2.4152
Epoch: 0/10............. Loss: 2.4964
Epoch: 0/10............. Loss: 2.3738
Epoch: 0/10............. Loss: 2.4090
Epoch: 0/10............. Loss: 2.5213
Epoch: 0/10............. Loss: 2.4293
Epoch: 0/10............. Loss: 2.4820
Epoch: 0/10............. Loss: 2.2655
Epoch: 0/10............. Loss: 2.4303
Epoch: 0/10............. Loss: 2.3805
Epoch: 0/10............. Loss: 2.2984
Epoch: 0/10............. Loss: 2.4218
Epoch: 0/10............. Loss: 2.2926
Epoch: 0/10............. Loss: 2.3243
Epoch: 0/10............. Loss: 2.3717
Epoch: 0/10............. Loss: 2.4067
Epoch: 0/10............. Loss: 2.3207
Epoch: 0/10.

In [151]:
torch.save(model.state_dict(), 'model_state_dict.pth')


In [166]:
model.load_state_dict(torch.load('model_state_dict.pth'))
model.eval()  # 设置模型为评估模式

text = "" 
text_length = len(text)
text_tokenize = tokenize(text, bidirectional=bidirectional) + [0] * 10 # 假设这是您的文本分词函数

texts_tokenize = [torch.tensor(text_tokenize, dtype=torch.long)]  # 创建一个包含您的文本的列表
padded_dataset = pad_sequence(texts_tokenize, batch_first=True, padding_value=0)  # 填充序列以创建统一长度的批次

# 创建隐藏状态，注意这里的 batch.shape[0] 应该替换为 padded_dataset的批次大小
hidden = model.init_hidden(padded_dataset.shape[0])

# 将数据和隐藏状态移到正确的设备（例如GPU）
padded_dataset = padded_dataset.to(device)

# 通过模型运行数据
output, hidden = model(padded_dataset, hidden)

max_values, max_indices = torch.max(output, dim=-1)
max_indices = max_indices.squeeze().tolist()
print(reverse_tokenize(max_indices))
name_a = text + reverse_tokenize(max_indices)[text_length-1:]
print(name_a)


eet
kurt
